# Intro to the Lowe Package

This notebook will introduce you how to use two of our core packages: `lowe.acs` and `lowe.locations`. First, make sure you can use the `lowe` conda environment as your jupyter kernel. To do that, run these two commands in your terminal (not in the notebook):

In [1]:
# conda install -c anaconda ipykernel
# python -m ipykernel install --user --name=lowe

After this, you can go to `Kernel > Change kernel > lowe` and you can switch your environment to the `lowe` environment in a notebook!

Since we install our project as a Python package, we can import our packages as follows:

In [1]:
import pandas as pd

from lowe.locations.lookup import name2fips, fips2name, search
from lowe.acs.ACSClient import ACSClient

# Using lowe.locations

If we want to make good use of the ACS wrapper, we need to encode locations in what are called **location dictionaries**. These are dictionaries where the keys are "state", "msa", "county", and "city". In practice, you will usually only use these combinations of keys:

- "state"
- "state", "msa"
- "state", "county"
- "state", "city"

That is, a state and a particular sub-geography. The **values** corresponding to these keys are **FIPS Codes**. FIPS stands for Federal Information Processing Standards. FIPS codes are unique codes corresponding to each geographical entity in the United states. How do we find these codes, though?

Enter `search()` from `lowe.locations.lookup`! This function takes 3 arguments:

- query (str): what you are searching for
- codetype (str): what geography type you are searching for (either "state", "msa", "county", or "city")
- search_on (str): what your query is (either "name" or "fips")

So, if we want to find the FIPS code, we might do

In [2]:
search(query = "palm springs", codetype = "city", search_on = "name")

                        name     fips
3087        palm springs, ca  0655254
5029        palm springs, fl  1254450
5030  palm springs north, fl  1254500


Note that we always pass everything in as lowercase!

So, the FIPS code for "palm springs, ca" is "0655254". The "06" corresponds to the state (CA), and the last 5 digits "55254" correspond to the city. Note that FIPS codes are **only** unique up to state! We can see this by searching for a FIPS code, say, "52540"

In [3]:
search(query = "52540", codetype = "city", search_on = "fips")

                   name     fips
12750      mayville, mi  2652540
16533        weston, ne  3152540
17798  north hurley, nm  3552540
25005       finklea, sc  4525405
25172        olanta, sc  4552540


So, only 7-digit (state-inclusive) FIPS codes are unique!

So, to query the ACS wrapper, we need to pass in a dictionary where the FIPS codes are values. So, we will pass in a dictionary that looks like this:

```python
loc = {
    "state": "06",
    "city": "55254" # the first two digits are optional if you pass in "state"
}
```

But the output of the ACS wrapper will translate the FIPS code to name values. How does it do this? Simply by calling `fips2name(loc)`:

In [4]:
loc = {
    "state": "06",
    "city": "55254" # If you pass in a 7-digit city code, then "state" is unnecessary
}

namedict = fips2name(loc)
print(namedict)

{'city': 'palm springs, ca', 'state': 'ca'}


Works like a charm! We can also reverse this by calling `fips2name(namedict)`:

In [5]:
name2fips(namedict)

{'city': '0655254', 'state': '06'}

Perfect :) 

Now that we have this under our belt, we are ready to use the ACS API wrapper!

# lowe.acs package

Now that we are armed with the locations package, let's try to make a query to the ACS API. 

First, what is ACS? This stands for the **American Community Survey**, which is published yearly by the US Census Bureau. The data are incredibly detailed -- take a look [here](https://data.census.gov/cedsci/). Click on `Advanced Search` and take a look at some of the categories they have!

Now, let's say we want to pull the poverty rates (table ID: [S1701](https://data.census.gov/cedsci/table?q=S1701&tid=ACSST5Y2019.S1701)) in Coachella, CA and Indian Wells, CA from 2012 to 2019 using ACS 5-year estimates (the most robust estimates they offer). First, we need to find the FIPS code for Coachella:

In [6]:
search(query="coachella, ca", codetype="city", search_on="name")

search(query="indian wells, ca", codetype="city", search_on="name")

               name     fips
2315  coachella, ca  0614260
                  name     fips
2681  indian wells, ca  0636434


With this in mind, we can create our location dictionary:

In [7]:
loc1 = {
    "state": "06",
    "city": "14260"
}

loc2 = {"city": "0636434"}

locs = [loc1, loc2]

In [8]:
client = ACSClient()
await client.initialize()

In [9]:
resp = await client.get_acs(vars = ["S1701"],
                     start_year="2012",
                     end_year="2019",
                     location=locs, # you can also pass in just one dictionary
                     estimate="5")

Now we can check out what columns there are and then process the result:

In [18]:
for col in list(resp.columns):
    print(col)

POVERTY STATUS IN THE PAST 12 MONTHS Estimate Below poverty level Population for whom poverty status is determined
POVERTY STATUS IN THE PAST 12 MONTHS Estimate Below poverty level Population for whom poverty status is determined AGE 18 to 64 years
POVERTY STATUS IN THE PAST 12 MONTHS Estimate Below poverty level Population for whom poverty status is determined AGE 18 to 64 years 18 to 34 years
POVERTY STATUS IN THE PAST 12 MONTHS Estimate Below poverty level Population for whom poverty status is determined AGE 18 to 64 years 35 to 64 years
POVERTY STATUS IN THE PAST 12 MONTHS Estimate Below poverty level Population for whom poverty status is determined AGE 60 years and over
POVERTY STATUS IN THE PAST 12 MONTHS Estimate Below poverty level Population for whom poverty status is determined AGE 65 years and over
POVERTY STATUS IN THE PAST 12 MONTHS Estimate Below poverty level Population for whom poverty status is determined AGE Under 18 years
POVERTY STATUS IN THE PAST 12 MONTHS Estimate

Let's get the poverty rate percent column and rename it:

In [19]:
col_sub = ["POVERTY STATUS IN THE PAST 12 MONTHS Estimate Percent below poverty level Population for whom poverty status is determined",
          "state", "city"]

In [20]:
resp = resp[col_sub]

In [21]:
resp = resp.rename(columns = {
    "POVERTY STATUS IN THE PAST 12 MONTHS Estimate Percent below poverty level Population for whom poverty status is determined": "perc_poverty"
})

In [22]:
pd.options.display.max_rows = 20
print(resp)

concept_label perc_poverty state          city
year                                          
2012                  27.9    ca     coachella
2012                   5.3    ca  indian wells
2013                  30.9    ca     coachella
2013                   5.2    ca  indian wells
2014                  31.5    ca     coachella
2014                   5.6    ca  indian wells
2015                  31.6    ca     coachella
2015                   4.6    ca  indian wells
2016                  30.1    ca     coachella
2016                   4.9    ca  indian wells
2017                  28.0    ca     coachella
2017                   4.4    ca  indian wells
2018                  23.4    ca     coachella
2018                   6.8    ca  indian wells
2019                  21.8    ca     coachella
2019                   6.7    ca  indian wells
